## TODO

In [ ]:
# PLAN MONDAY 
# FIX Model input Data 
# TODO Create equivalent variables but flip nodes with edges 

# DONE Understand architecture 
# DONE Understand model input data 
# DONE There are missing connections in the adjacency matrix # IDEA scrap adjacency matrix entirely if necessary? 
# DONE Create model without NeighborSampler 
# DONE Clean code 
# DONE Make model work ... 

# PLAN TUESDAY 
# TODO ADMIN - Send email with Update (ask about servers?) 
# TODO CODE - Make model more extensive 
# TODO CODE - Visualize prediction patterns and learning curve for existing model 
# TODO CODE - Create testing stage and evaluation 
# TODO THEORY - Watch GNN Theory 
# TODO THEORY - Write up Introduction 

# PLAN WEDNESDAY 
# TODO THEORY - Understand Math Theory 
# TODO THEORY - Write up methodology theory 
# TODO THEORY - Write up Literature Review 

# PLAN THURSDAY 
# TODO CODE - add statistics to a new DataFrame 
# TODO CODE - add average of transactions for accounts 

# GRAPH NEURAL NETWORKS

In [ ]:
# Define the neural network model
# class Net(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels, heads):
#         super(Net, self).__init__()
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, out_channels)
#         self.fc = nn.Linear(out_channels, 1)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index)
#         x = torch.relu(x)
#         x = self.conv2(x, edge_index)
#         x = torch.relu(x)
#         x = self.fc(x)
#         return x

class Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
print(input_data.x.size())
print(input_data.y.size())

torch.Size([4188, 31])
torch.Size([4999])


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define hyperparameters
num_epochs = 2000
hidden_channels = 128
out_channels = 64
learning_rate = 0.01

# Initialize the model, optimizer, and loss function
# model = LinkPredictor(in_channels=input_data.num_features, hidden_channels=64, out_channels=32, heads=8).to(device)
# optimizer = optim.SGD(params=model.parameters(), lr=learning_rate) # IDEA SGD = Stochastic Gradient Descent
# # criterion = torch.nn.BCELoss()
# criterion = nn.BCEWithLogitsLoss() # IDEA used for binary classification tasks --> SIGMOID + BINARY CROSS-ENTROPY LOSS

# Initialize the model, optimizer, and loss function
model = Net(in_channels=node_features.size(1), hidden_channels=64, out_channels=32).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

# Define the data loader
loader = DataLoader([input_data], batch_size=1, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_fraudulent_edges = 0  # Initialize total count of fraudulent edges
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        
        # Reshape the output tensor to match the shape of the target tensor
        out = out.view(-1)  # Flatten the output tensor
        
        # Ensure the dimensions of out and batch.y match
        if out.size(0) != batch.y.size(0):
            batch_size = batch.y.size(0)
            out = out[:batch_size]  # Trim the output tensor to match the batch size
        
        # Compute the binary cross-entropy loss
        loss = criterion(out, batch.y.float())  # Ensure batch.y is float type
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        # Predictions: convert logits to probabilities using sigmoid
        probabilities = torch.sigmoid(out)
        
        # Threshold probabilities to predict fraud (1) or non-fraud (0)
        predicted_labels = (probabilities > 0.5).int()  # Threshold at 0.5
        
        # Count the number of fraudulent edges (predicted as 1)
        fraudulent_edges_in_batch = (predicted_labels == 1).sum().item()
        total_fraudulent_edges += fraudulent_edges_in_batch
    
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Fraudulent Edges: {total_fraudulent_edges}')


Epoch 1, Loss: 0.7222440838813782, Fraudulent Edges: 2675
Epoch 2, Loss: 0.6448321342468262, Fraudulent Edges: 1640
Epoch 3, Loss: 0.5735844373703003, Fraudulent Edges: 880
Epoch 4, Loss: 0.49813541769981384, Fraudulent Edges: 423
Epoch 5, Loss: 0.4165233075618744, Fraudulent Edges: 163
Epoch 6, Loss: 0.3318313658237457, Fraudulent Edges: 33
Epoch 7, Loss: 0.25025108456611633, Fraudulent Edges: 3
Epoch 8, Loss: 0.17823021113872528, Fraudulent Edges: 0
Epoch 9, Loss: 0.12013284862041473, Fraudulent Edges: 0
Epoch 10, Loss: 0.07724067568778992, Fraudulent Edges: 0
Epoch 11, Loss: 0.04795978590846062, Fraudulent Edges: 0
Epoch 12, Loss: 0.029194707050919533, Fraudulent Edges: 0
Epoch 13, Loss: 0.017733199521899223, Fraudulent Edges: 0
Epoch 14, Loss: 0.010942677035927773, Fraudulent Edges: 0
Epoch 15, Loss: 0.0069821602664887905, Fraudulent Edges: 0
Epoch 16, Loss: 0.004688794258981943, Fraudulent Edges: 0
Epoch 17, Loss: 0.003361760638654232, Fraudulent Edges: 0
Epoch 18, Loss: 0.0025913

In [ ]:
# Plot training loss
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.show()

# Compute statistics of test AUC values
mean_auc = np.mean(test_aucs)
median_auc = np.median(test_aucs)
std_auc = np.std(test_aucs)
min_auc = np.min(test_aucs)
max_auc = np.max(test_aucs)

# Print the statistics
print(f"Mean Test AUC: {mean_auc:.4f}")
print(f"Median Test AUC: {median_auc:.4f}")
print(f"Standard Deviation of Test AUC: {std_auc:.4f}")
print(f"Minimum Test AUC: {min_auc:.4f}")
print(f"Maximum Test AUC: {max_auc:.4f}")


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling, train_test_split_edges
import networkx as nx

# Load your DataFrame and extract required columns (e.g., node features and labels)
# Assuming node_features and labels are numpy arrays

# Define the graph
graph_full = nx.Graph()

# Add all links to the graph
for i in range(len(links)):
    u = links[i].get("source")
    v = links[i].get("destination")
    graph_full.add_edge(u, v, label=edges_amount[i])

# Convert adjacency matrix to edge_index tensor
adjacency_matrix = nx.adjacency_matrix(graph_full)
num_nodes = adjacency_matrix.shape[0]
edge_index = torch.tensor(np.array(adjacency_matrix.nonzero()), dtype=torch.long)

# For undirected graphs, concatenate the edge_index with its reverse
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

# Define node features and labels
node_features = torch.tensor(x, dtype=torch.float)
labels = torch.tensor(labels, dtype=torch.float)  # Assuming y contains labels for each transaction

# Create the PyTorch Geometric Data object
data = Data(x=node_features, edge_index=edge_index, y=labels)

# Separate dataset into training and testing sets
data = train_test_split_edges(data)

# Define hyperparameters
num_epochs = 100
hidden_channels = 64
out_channels = 32
learning_rate = 0.01

# Define the GCN model
class Net(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.fc = nn.Linear(out_channels, 1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.fc(x)
        return x

# Initialize the model, optimizer, and loss function
model = Net(node_features.shape[1], hidden_channels, out_channels).to(device)
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss()

# Modify train function to use all nodes in the graph for target labels
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x.to(device), data.train_pos_edge_index.to(device))
    # Flatten the output and target tensors
    out_flat = out.view(-1)
    target = data.y.view(-1)
    loss = criterion(out_flat, target.to(device))
    loss.backward()
    optimizer.step()
    return loss.item()

# Modify test function to predict labels for all nodes in the graph
def test(data):
    model.eval()
    with torch.no_grad():
        out = model(data.x.to(device), data.train_pos_edge_index.to(device))
        return out.cpu().numpy()

# Train/Test Loop with logging
best_val_auc = final_test_auc = 0
for epoch in range(1, num_epochs + 1):
    loss = train(data)
    test_auc = test(data)
    print(f'Epoch: {epoch}, Loss: {loss}, Test AUC: {test_auc}')


In [ ]:
# Test all instances
predictions_all, probabilities_all = test(data)

# Count TRUE and FALSE predictions
num_true = np.sum(predictions_all)
num_false = len(predictions_all) - num_true
print(f"Number of TRUE predictions: {num_true}")
print(f"Number of FALSE predictions: {num_false}")

# Inspect specific instances (e.g., indices 0, 1, 2)
indices_to_inspect = [0, 1, 2]
predictions_specific, probabilities_specific = test(data, indices=indices_to_inspect)
print("Predictions for specific instances:", predictions_specific)
print("Probabilities for specific instances:", probabilities_specific)

In [ ]:
import torch
from model import GAT
from dataset import AMLtoGraph
import torch_geometric.transforms as T
from torch_geometric.loader import NeighborLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = AMLtoGraph('/path/to/AntiMoneyLaunderingDetectionWithGNN/data')
data = dataset[0]
epoch = 100

model = GAT(in_channels=data.num_features, hidden_channels=16, out_channels=1, heads=8)
model = model.to(device)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

split = T.RandomNodeSplit(split='train_rest', num_val=0.1, num_test=0)
data = split(data)

train_loader = NeighborLoader(
    data,
    num_neighbors=[30] * 2,
    batch_size=256,
    input_nodes=data.train_mask,
)

test_loader = NeighborLoader(
    data,
    num_neighbors=[30] * 2,
    batch_size=256,
    input_nodes=data.val_mask,
)

for i in range(epoch):
    total_loss = 0
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        data.to(device)
        pred = model(data.x, data.edge_index, data.edge_attr)
        ground_truth = data.y
        loss = criterion(pred, ground_truth.unsqueeze(1))
        loss.backward()
        optimizer.step()
        total_loss += float(loss)
    if epoch%10 == 0:
        print(f"Epoch: {i:03d}, Loss: {total_loss:.4f}")
        model.eval()
        acc = 0
        total = 0
        with torch.no_grad():
            for test_data in test_loader:
                test_data.to(device)
                pred = model(test_data.x, test_data.edge_index, test_data.edge_attr)
                ground_truth = test_data.y
                correct = (pred == ground_truth.unsqueeze(1)).sum().item()
                total += len(ground_truth)
                acc += correct
            acc = acc/total
            print('accuracy:', acc)

In [ ]:
# node embedding: ADJACENCY MATRIX * NODE FEATURES

In [ ]:
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the GNN architecture
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.gc1 = GraphConvolution(input_dim, hidden_dim)
        self.gc2 = GraphConvolution(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, adj):
        x = self.relu(self.gc1(x, adj))
        x = self.relu(self.gc2(x, adj))
        x = self.fc(x)
        return x

class GraphConvolution(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraphConvolution, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x, adj):
        out = torch.matmul(adj, x)  # Aggregate neighbor information
        out = self.linear(out)
        return out


In [ ]:
# Prepare data
# Assuming x is node features, y is edge features, and adjacency_matrix is the adjacency matrix

# Define some constants
num_nodes = x.shape[0]
input_dim = x.shape[1]
output_dim = 1  # Binary classification: fraud or not
hidden_dim = 64

# Convert numpy arrays to PyTorch tensors
x = torch.FloatTensor(x)
y = torch.FloatTensor(y)
adjacency_matrix = torch.FloatTensor(adjacency_matrix) ### INEFFICIENT !!!

# Split data into train and test sets
num_train_samples = 4000
train_indices = np.random.choice(num_nodes, num_train_samples, replace=False)
test_indices = np.setdiff1d(np.arange(num_nodes), train_indices)

x_train, x_test = x[train_indices], x[test_indices]
adj_train, adj_test = adjacency_matrix[train_indices][:, train_indices], adjacency_matrix[test_indices][:, test_indices]
y_train, y_test = y[train_indices], y[test_indices]

# Create DataLoader
train_dataset = TensorDataset(x_train, adj_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the model
model = GNN(input_dim, hidden_dim, output_dim)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        x_batch, adj_batch, y_batch = batch
        output = model(x_batch, adj_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss}")


In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    output = model(x_test, adj_test)
    predictions = torch.sigmoid(output).round().squeeze().numpy()

# Assuming y_test contains the true labels
accuracy = np.mean(predictions == y_test.numpy())
print(f"Accuracy: {accuracy}")


## Creating Learnable Node Embeddings

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, adj_matrix, node_features):
        # Convert adjacency matrix to PyTorch tensor
        adj_matrix_tensor = torch.from_numpy(adj_matrix).float()
        # Normalize adjacency matrix
        adj_matrix_normalized = torch.matmul(adj_matrix_tensor, torch.diag(torch.pow(torch.sum(adj_matrix_tensor, dim=1), -0.5)))
        # Convert node features to PyTorch tensor
        node_features_tensor = torch.from_numpy(node_features).float()
        # Compute node embeddings
        node_embeddings = torch.matmul(adj_matrix_normalized, node_features_tensor)
        # Apply linear transformation
        output = self.linear(node_embeddings)
        # Apply ReLU activation function
        output = F.relu(output)
        return output

class GCN(nn.Module):
    def __init__(self, num_features, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.layer1 = GCNLayer(num_features, hidden_size)
        self.layer2 = GCNLayer(hidden_size, num_classes)

    def forward(self, adj_matrix, node_features):
        h1 = self.layer1(adj_matrix, node_features)
        output = self.layer2(adj_matrix, h1)
        return output


In [ ]:

# Example usage:
num_features = 10  # Number of node features
hidden_size = 16   # Hidden layer size
num_classes = 2    # Number of output classes
adj_matrix = torch.randn(10, 10)  # Example adjacency matrix
node_features = torch.randn(10, num_features)  # Example node features

# Create GCN model
model = GCN(num_features, hidden_size, num_classes)

# Forward pass
output = model(adjacency_matrix, x)


## SIMPLE GNN MODEL EXAMPLE

In [ ]:
import torch
import torch.nn as nnd1
import torch.nn.functional as F

In [ ]:
num_features =
hidden_dim =
num_classes =
num_epochs =
features =
adj =
labels =

In [ ]:
class GCNBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNBlock, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, adj):
        x = self.linear(x)
        x = torch.matmul(adj, x)
        x = F.relu(x)
        return x
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gcn1 = GCNBlock(input_dim, hidden_dim)
        self.gcn2 = GCNBlock(hidden_dim, output_dim)

    def forward(self, x, adj):
        x = self.gcn1(x, adj)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.gcn2(x, adj)
        return x
# Define the model
model = GCN(num_features, hidden_dim, num_classes)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# Train the model
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(features, adj)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()